<a href="https://colab.research.google.com/github/purplepatch-everyday/ImageClassification/blob/main/Basic_Conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
# 하이퍼파라미터 설정
batch_size = 64
epochs = 10
learning_rate = 0.001

In [3]:
# 데이터 로드 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 266222561.74it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 54247959.62it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 220560383.87it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10842645.86it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Convolutional Neural Network (CNN) 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(32 * 7 * 7, 10)  # MNIST 이미지 크기가 28x28이므로 2번 pooling 후에 7x7 크기가 됩니다.

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten 작업
        x = self.fc(x)
        return x

# 모델 생성 및 손실함수, 옵티마이저 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 훈련(training) 함수 정의
def train(model, criterion, optimizer, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}")

# 테스트(test) 함수 정의
def test(model, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100.0 * correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")

# 모델 훈련 및 테스트 수행
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train(model, criterion, optimizer, train_loader)
    test(model, criterion, test_loader)

Epoch 1/10
Batch [0/938], Loss: 2.3061981201171875
Batch [100/938], Loss: 0.17681927978992462
Batch [200/938], Loss: 0.18906323611736298
Batch [300/938], Loss: 0.12445729225873947
Batch [400/938], Loss: 0.2360791712999344
Batch [500/938], Loss: 0.035687871277332306
Batch [600/938], Loss: 0.016255568712949753
Batch [700/938], Loss: 0.1371689736843109
Batch [800/938], Loss: 0.020269159227609634
Batch [900/938], Loss: 0.12454476952552795
Test Loss: 0.0009, Accuracy: 98.10%
Epoch 2/10
Batch [0/938], Loss: 0.022075586020946503
Batch [100/938], Loss: 0.05123133957386017
Batch [200/938], Loss: 0.19920802116394043
Batch [300/938], Loss: 0.012456205673515797
Batch [400/938], Loss: 0.08167731761932373
Batch [500/938], Loss: 0.05365363880991936
Batch [600/938], Loss: 0.019564596936106682
Batch [700/938], Loss: 0.01990361139178276
Batch [800/938], Loss: 0.029724525287747383
Batch [900/938], Loss: 0.11014459282159805
Test Loss: 0.0008, Accuracy: 98.33%
Epoch 3/10
Batch [0/938], Loss: 0.018688328564